In [1]:
import pandas as pd
import ipynb.fs.full.features as features_nltk

# df_train = pd.read_csv('./data/train.csv')
# df_test = pd.read_csv('./data/test.csv')

In [2]:
def concatenar_categoricos(df, columna1, columna2, N=0, df_aux=None, devolver_cols=False):
    '''Concatena dos columnas categoricas y hace one hot'''
    '''Ver referencia columna_a_ohe para los ultimos 3 parametros'''
    
    def limpiar_espacios_blanco(texto):
        try:
            return texto.replace(' ', '_')
        except:
            pass
        
    df = df.copy()
    df[columna1] = df[columna1].apply(limpiar_espacios_blanco)
    df[columna2] = df[columna2].apply(limpiar_espacios_blanco)
    df[columna1 + '_' + columna2] = df[columna1] + '_' +df[columna2]
    
    features_nltk
    return features_nltk.columna_a_ohe(df, columna1 + '_' + columna2, N, df_aux, devolver_cols)

In [3]:
def cantidad_propiedad_misma_cantidad(df, agrupar_col, misma_cantidad_col):
    '''Cantidad de casas con la misma cantidad de banos que la actual'''
    '''Por ejemplo, retorna para cada propiedad, cuantas hay mas de su tipo con x banos ej.'''
    '''cantidad_propiedad_misma_cantidad(df, 'banos', 'tipodepropiedad'), para cada propiedad, se fija
    cuantas mas de su tipo con tantos banos hay'''
    '''Retorna el df con la columna agregada'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([agrupar_col, misma_cantidad_col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    def get_cantidad(col1, col2):
        cantidad = df_aux[(df_aux[agrupar_col] == col1) & (df_aux[misma_cantidad_col] == col2)]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
        
    df['cantidad_' + agrupar_col + '_' + misma_cantidad_col] = df.apply(lambda x: get_cantidad(x[agrupar_col], x[misma_cantidad_col]), axis=1)
    return df

In [4]:
def cantidad_atributo(df, col):
    '''Simil a value_counts. Dada una columna, agrega al df, cuantas propiedades existen con esa misma propiedad'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()

    def get_cantidad(col1):
        cantidad = df_aux[df_aux[col] == col1]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
    
    df['cantidad_propiedades_en_' + col] = df.apply(lambda x: get_cantidad(x[col]), axis=1)
    return df

In [5]:
def ranking_atributo(df, col):
    '''Cuenta cuantas propiedades hay con los atributos de la columna recibida. Y para cada propiedad,
    se fija en que posicion esta del ranking. Por ejemplo, si hay dos propuedades, una con dos banos y otra 
    con uno solo, el rankin de las que tengan dos banos en 1, y de las propiedades que tengan un solo bano es 
    dos'''
    
    '''Rellena los nulos con "otros" '''
    df[col] = df[col].fillna('otros')
    
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    df_aux = df_aux.nlargest(df_aux.shape[0], 'cantidad') 
    lista_ranking = df_aux[col].tolist()
    rank = {}
    contador = 1
    
    for item in lista_ranking:
        rank[item] = contador
        contador = contador + 1

    def get_ranking(col1):
        item = df_aux[df_aux[col] == col1][col].values[0]
        return rank[item]
    
    df['ranking_en_' + col] = df.apply(lambda x: get_ranking(x[col]), axis=1)
    return df

In [6]:
def aplicar_features(df_recibido):
    df = df_recibido.copy()    
    df = concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
    df = concatenar_categoricos(df, 'provincia', 'ciudad')
    # -- 
    display('Check-1.0')
    df = cantidad_atributo(df, 'provincia')
    display('Check-1.1')
    df = cantidad_atributo(df, 'ciudad')
    display('Check-1.2')
    df = cantidad_atributo(df, 'antiguedad')
    display('Check-1.3')
    df = cantidad_atributo(df, 'habitaciones')
    display('Check-1.4')
    df = cantidad_atributo(df, 'garages')
    display('Check-1.5')
    df = cantidad_atributo(df, 'banos')
    display('Check-1.6')
    df = cantidad_atributo(df, 'gimnasio')
    display('Check-1.7')
    df = cantidad_atributo(df, 'usosmultiples')
    display('Check-1.8')
    df = cantidad_atributo(df, 'escuelascercanas')
    display('Check-1.9')
    df = cantidad_atributo(df, 'centroscomercialescercanos')
    display('Check-1.10')
    df = cantidad_atributo(df, 'piscina')
    # --
    display('Check-2.0')
    df = ranking_atributo(df, 'provincia')
    display('Check-2.1')
    df = ranking_atributo(df, 'tipodepropiedad')
    display('Check-2.2')
    df = ranking_atributo(df, 'ciudad')
    display('Check-2.3')
    df = ranking_atributo(df, 'antiguedad')
    display('Check-2.4')
    df = ranking_atributo(df, 'habitaciones')
    display('Check-2.5')
    df = ranking_atributo(df, 'garages')
    display('Check-2.6')
    df = ranking_atributo(df, 'banos')
    display('Check-2.7')
    df = ranking_atributo(df, 'gimnasio')
    display('Check-2.8')
    df = ranking_atributo(df, 'usosmultiples')
    display('Check-2.9')
    df = ranking_atributo(df, 'escuelascercanas')
    display('Check-2.10')
    df = ranking_atributo(df, 'centroscomercialescercanos')    
    # --
    display('Check-3.0')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
    display('Check-3.1')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'ciudad')
    display('Check-3.2')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'antiguedad')
    display('Check-3.3')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'habitaciones')
    display('Check-3.4')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'garages')
    display('Check-3.5')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'banos')
    display('Check-3.6')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'gimnasio')
    display('Check-3.7')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'usosmultiples')
    display('Check-3.8')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'piscina')
    display('Check-4.0')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'antiguedad')
    display('Check-4.1')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'habitaciones')
    display('Check-4.2')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'garages')
    display('Check-4.3')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'banos')
    display('Check-4.4')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'gimnasio')
    display('Check-4.5')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'usosmultiples')
    display('Check-5.6')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'antiguedad')
    display('Check-5.7')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'habitaciones')
    display('Check-5.8')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'garages')
    display('Check-5.9')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'banos')
    display('Check-5.10')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'gimnasio')
    display('Check-5.11')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'usosmultiples')
    display('Check-5.12')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'piscina')
    display('Check-6.0')
    return df

In [7]:
df = df_train
# df = df_test

display(df.columns)
display(df.shape)

#Ejs:
# concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
# cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
# cantidad_atributo(df, 'provincia')
# ranking_atributo(df, 'provincia')

df = aplicar_features(df)
# df.to_csv('masfeatures_train.csv', index=False)
# df.to_csv('masfeatures_test.csv', index=False)

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

(240000, 23)

'Check-1.0'

'Check-1.1'

'Check-1.2'

...

'Check-5.9'

'Check-5.10'

'Check-5.11'

'Check-5.12'

'Check-6.0'

In [3]:
df_train = pd.read_csv('./masfeatures_train.csv')
df_eval = pd.read_csv('./masfeatures_test.csv')


# Correccion csv - strings que deberian ser floats
df_train['antiguedad'] = df_train['antiguedad'].apply(lambda antiguedad: None if antiguedad == 'otros' else antiguedad)
df_eval['antiguedad'] = df_eval['antiguedad'].apply(lambda antiguedad: None if antiguedad == 'otros' else antiguedad)
df_eval['antiguedad'] = df_eval['antiguedad'].astype(float)
df_train['antiguedad'] = df_train['antiguedad'].astype(float)

df_train['habitaciones'] = df_train['habitaciones'].apply(lambda x: None if x == 'otros' else x)
df_eval['habitaciones'] = df_eval['habitaciones'].apply(lambda x: None if x == 'otros' else x)
df_eval['habitaciones'] = df_eval['habitaciones'].astype(float)
df_train['habitaciones'] = df_train['habitaciones'].astype(float)

df_train['garages'] = df_train['garages'].apply(lambda x: None if x == 'otros' else x)
df_eval['garages'] = df_eval['garages'].apply(lambda x: None if x == 'otros' else x)
df_eval['garages'] = df_eval['garages'].astype(float)
df_train['garages'] = df_train['garages'].astype(float)

df_train['banos'] = df_train['banos'].apply(lambda x: None if x == 'otros' else x)
df_eval['banos'] = df_eval['banos'].apply(lambda x: None if x == 'otros' else x)
df_eval['banos'] = df_eval['banos'].astype(float)
df_train['banos'] = df_train['banos'].astype(float)

# Las columnas de tipo object
x = df_train.columns.to_series().groupby(df_train.dtypes).groups
display("Las columnas de tipo Object:")
display(list(x[list(x.keys())[len(list(x.keys()))-1]]))

df_eval.to_csv('masfeatures_train_f.csv', index=False)
df_train.to_csv('masfeatures_test_f.csv', index=False)

'Las columnas de tipo Object:'

['titulo',
 'descripcion',
 'tipodepropiedad',
 'direccion',
 'ciudad',
 'provincia',
 'fecha',
 'tipodepropiedad_provincia',
 'provincia_ciudad']